In [1]:
#필요 모듈 import
from google.colab import drive
drive.mount('/content/drive')
import io
import os
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import math
import itertools

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
archive_path = '/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data'
#임의로 나눈 파일 디렉토리 읽어서 각 csv 파일 제목 저장
study_file=os.listdir('/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data')
print(study_file)
test_file = os.listdir('/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/test_data')
print(test_file)

['abortion_counts.csv', 'blm_movement_counts.csv', 'islam_counts.csv', 'espionage_counts.csv', 'human_rights_counts.csv', 'china_counts.csv', 'judaism_counts.csv', 'military_spending_counts.csv', 'guns_counts.csv', 'sexual_harrassment_counts.csv', 'oil_counts.csv', 'russia_counts.csv', 'native_americans_counts.csv', 'palestine_counts.csv', 'tech_censorship_counts.csv', 'universities_counts.csv']
['affirmative_action_counts.csv', 'israel_counts.csv', 'church_state_counts.csv', 'climate_change_counts.csv', 'cuba_counts.csv', 'prisons_counts.csv', 'public_finance_counts.csv', 'nuclear_weapons_counts.csv', 'police__counts.csv', 'private_finance_counts.csv', 'yemen_counts.csv', 'venezuela_counts.csv', 'us_immigration_counts.csv']


In [3]:
#study_data 파일 읽어오기
%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data
for i in range(0, len(study_file)-1):
  globals()['study_data{}'.format(i)] = pd.read_csv(study_file[i], encoding='cp949', header=None)
  globals()['study_data{}'.format(i)] = globals()['study_data{}'.format(i)].drop(1, axis=1)
# print(globals()['study_data{}'.format(0)].head)

#test_data 파일 읽어오기
%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/test_data
for i in range(0, len(test_file)-1):
  globals()['test_data{}'.format(i)] = pd.read_csv(test_file[i], encoding='cp949', header=None)
  globals()['test_data{}'.format(i)] = globals()['test_data{}'.format(i)].drop(1, axis=1)

/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/study_data
/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/phrase_counts/test_data


In [4]:
#단어 리스트 
for i in range(0, len(study_file)-1):
  a = globals()['study_data{}'.format(i)][0]
  globals()['st_word{}'.format(i)] = a.values.tolist()
  del globals()['st_word{}'.format(i)][0]

# print(globals()['st_word{}'.format(0)])

#언론사 리스트
for i in range(0, len(study_file)-1):
  globals()['media_list{}'.format(i)] = globals()['study_data{}'.format(i)].iloc[0].tolist()
  del globals()['media_list{}'.format(i)][0]
# print(media_list0)
# print(len(media_list0))

In [5]:
#최대 단어 3개 뽑을때 사용할 csv 전처리(column, row 각각 언론사와 단어로 지정해 새로 만듬)
#런타임 내에서 다시 실행시 st_d 데이터 프레임 이상 생기므로 실행시 런타임 초기화 필요
for i in range(0, len(study_file)-1):
  globals()['st_d{}'.format(i)]=globals()['study_data{}'.format(i)].drop(0).drop(0,axis=1)
  globals()['st_d{}'.format(i)]=globals()['st_d{}'.format(i)].apply(pd.to_numeric)
  globals()['st_d{}'.format(i)].columns = globals()['study_data{}'.format(i)].iloc[0,1:]
  globals()['st_d{}'.format(i)].reset_index(drop=True)
  globals()['st_d{}'.format(i)]=globals()['st_d{}'.format(i)].set_index(globals()['study_data{}'.format(i)].iloc[1:,0])

# print(len(globals()['st_d{}'.format(0)]))
# globals()['st_d{}'.format(0)]


In [6]:
#각 파일-각 언론사 별 자주 사용하는 3개 단어(리스트로 저장)

for k in range(0, len(study_file)-1):
  globals()['word_3{}'.format(k)]=[]
  for i in range(0, len(globals()['media_list{}'.format(k)])-1):
    df = globals()['st_d{}'.format(k)]
    a = df.nlargest(3, globals()['media_list{}'.format(k)][i])  #가장 높은 값 가진 3개 숫자를 데이터 프레임으로 생성.
    A = a.index.to_list() #a(데이터프레임) index만 추출해 리스트로 변환.
    # print(A)
    globals()['word_3{}'.format(k)].append(A) #A에 있는 index 리스트 추가.

# print(len(globals()['word_3{}'.format(0)]))
# globals()['word_3{}'.format(0)]

In [7]:
#3차원 시각화

for k in range(0, len(study_file)-1):
  data = globals()['word_3{}'.format(k)]
  mapping = {item: idx for idx, item in enumerate(globals()['st_word{}'.format(k)])}
  # print(mapping)

  mapped_data = [[mapping[val] for val in lst] for lst in data]
  # print(mapped_data)

  data_int = []
  for point in data:
      point_int = [mapping[label] for label in point]
      data_int.append(point_int)

  # 축 레이블 리스트
  axis_labels = list(mapping.keys())

  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(111, projection='3d')

  # 축 범위 지정
  ax.set_xlim(0, len(mapping) - 1)
  ax.set_ylim(0, len(mapping) - 1)
  ax.set_zlim(0, len(mapping) - 1)


  # 축 레이블 지정
  ax.set_xticks(range(len(axis_labels)))
  ax.set_yticks(range(len(axis_labels)))
  ax.set_zticks(range(len(axis_labels)))
  ax.set_xticklabels(axis_labels, fontsize=4, rotation=90)
  ax.set_yticklabels(axis_labels, fontsize=4, rotation=0)
  ax.set_zticklabels(axis_labels, fontsize=4, rotation=0)

  # 데이터 플롯
  x = [point[0] for point in data_int]
  y = [point[1] for point in data_int]
  z = [point[2] for point in data_int]
  ax.scatter(x, y, z)

  labels = globals()['media_list{}'.format(k)]

  for i, label in enumerate(labels[:len(data_int)]):
      ax.text(x[i], y[i], z[i], label, fontsize=6)
  filename=f'/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/study_image/image_{k}.png'
  plt.savefig(filename, dpi=300)
  # plt.show()
  

Output hidden; open in https://colab.research.google.com to view.

In [8]:
#저장을 위한 경로 변경
%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze/study_distance/

#거리 계산 데이터프레임 csv 저장
for k in range(0, len(study_file)-1):

  data = globals()['word_3{}'.format(k)]
  mapping = {item: idx for idx, item in enumerate(globals()['st_word{}'.format(k)])}
  media_list = globals()['media_list{}'.format(k)]
  print(mapping)

  # 새로운 데이터프레임 행, 열 인덱스 지정
  columns = media_list
  index = media_list

  mapped_data = [[mapping[val] for val in lst] for lst in data]

  data_int = []
  for point in data:
      point_int = [mapping[label] for label in point]
      data_int.append(point_int)

  distances = []
  for i, pair in enumerate(itertools.combinations(data_int, 2)):
      x1, y1, z1 = pair[0]
      x2, y2, z2 = pair[1]
      distance = ((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)**0.5
      distances.append(distance)

  df = pd.DataFrame(columns=columns, index=index)
  for i in range(len(media_list)):
      for j in range(len(media_list)):
          if i == j:
              df.iloc[i, j] = np.nan
          elif i > j:
              idx = 3*(j-1) - (j-1)*(j-2)//2 + (i-j-1)
              df.iloc[i, j] = distances[idx]
          else:
              idx = 3*(i-1) - (i-1)*(i-2)//2 + (j-i-1)
              df.iloc[i, j] = distances[idx]

  # display(df)
  globals()['distance{}'.format(k)]=df
  file_name = study_file[k] + 'distance.csv'
  df.to_csv(file_name, index=True)
  # print(f"{file_name} 파일이 저장되었습니다.")


/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/study_distance
{'pro life': 0, 'abortion rights': 1, 'anti abortion': 2, 'anti choice': 3, 'abortion care': 4, 'pregnancy': 5, 'kill': 6, 'medical abortion': 7, 'family planning': 8, 'pro abortion': 9, 'murder': 10, 'fetus': 11, 'mother': 12, 'reproductive rights': 13, 'infanticide': 14, 'abortion access': 15, 'abortion ban': 16, 'infant': 17, 'reproductive health': 18, 'pro choice': 19, 'abortion law': 20, 'medication abortion': 21, 'contraception': 22, 'fetal heartbeat': 23, 'fetal remains': 24, 'abortion services': 25, 'born alive': 26, 'birth control': 27, 'religious liberty': 28, 'aborted': 29, 'abortion rules': 30, 'abortion providers': 31, 'reproductive justice': 32, 'womens rights': 33, 'dead': 34, 'termination': 35, 'abortion clinic': 36, 'adoption': 37, 'rape or incest': 38, 'maternal': 39, 'abortion restrictions': 40, 'abortion pills': 41, 'unborn babies': 42, 'abortionists': 43, 'fetal abnormalities': 44, 'human life'

In [22]:
#거리 계산 데이터 프레임 통한 정확도 산출

%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze
# 데이터프레임의 개수
num_frames = len(study_file) - 1

# 새로운 데이터프레임 생성
mean_df = pd.DataFrame(index=columns, columns=index)

# 각 위치의 평균값 계산
for i in range(len(media_list)):
    for j in range(len(media_list)):
        values = []
        for k in range(num_frames):
            try:
                value = globals()['distance{}'.format(k)].iloc[i, j]
                if pd.notnull(value):
                    values.append(value)
            except IndexError:
                continue
        mean_value = np.mean(values) if values else np.nan
        mean_df.iloc[i, j] = mean_value

# 새로운 데이터프레임 저장
mean_df.to_csv('mean_distance.csv', index=True)
print("평균 거리 데이터프레임이 저장되었습니다.")



/content/drive/MyDrive/Colab Notebooks/3113_data_analyze
평균 거리 데이터프레임이 저장되었습니다.


In [23]:
# 거리 계산
distances = []
for i, point1 in enumerate(data_int):
    max_distance = 0
    for j, point2 in enumerate(data_int):
        if i != j:
            distance = ((point1[0] - point2[0]) ** 2 +
                        (point1[1] - point2[1]) ** 2 +
                        (point1[2] - point2[2]) ** 2) ** 0.5
            if distance > max_distance:
                max_distance = distance
    distances.append(max_distance)

# 가장 먼 거리 출력
max_distance = max(distances)
print("가장 먼 거리:", max_distance)


가장 먼 거리: 43.22036556994862


In [37]:
# 유사도 계산 함수
def calculate_similarity(distances, max_distance):
    similarities = [1 - (distance / max_distance) for distance in distances]
    return similarities

# 저장 경로 설정
%cd /content/drive/MyDrive/Colab Notebooks/3113_data_analyze/similar_study/

# 데이터프레임 저장
for k in range(0, len(study_file)-1):
    distance_df = globals()['distance{}'.format(k)]

    # 거리 계산
    distances = []
    for i, point1 in enumerate(data_int):
        max_distance = 0
        for j, point2 in enumerate(data_int):
            if i != j:
                distance = ((point1[0] - point2[0]) ** 2 +
                            (point1[1] - point2[1]) ** 2 +
                            (point1[2] - point2[2]) ** 2) ** 0.5
                if distance > max_distance:
                    max_distance = distance
        distances.append(max_distance)

    # 최대 거리 계산
    max_distance = max(distances)

    # 유사도 계산
    similarities = calculate_similarity(distances, max_distance)

    # 백분율 표현
    similarities_percent = [similarity * 100 for similarity in similarities]

    # 데이터프레임 생성
    similarity_df = pd.DataFrame(similarities_percent, columns=['Similarity'], index=distance_df.index)

    # 파일 저장
    similarity_file = study_file[k] + 'similarity.csv'
    similarity_df.to_csv(similarity_file, index=True)
    print(f"{similarity_file} 파일이 저장되었습니다.")


/content/drive/MyDrive/Colab Notebooks/3113_data_analyze/similar_study
abortion_counts.csvsimilarity.csv 파일이 저장되었습니다.


ValueError: ignored

In [24]:
#유사도 산출

similarity_df = pd.DataFrame(index=columns, columns=index)

# 각 위치의 유사도 계산
for i in range(len(media_list)):
    for j in range(len(media_list)):
        distance_value = mean_df.iloc[i, j]
        if pd.notnull(distance_value):
            similarity_value = 100 * (1 - distance_value / max_distance)
            similarity_df.iloc[i, j] = similarity_value
        else:
            similarity_df.iloc[i, j] = np.nan

# 새로운 데이터프레임 저장
similarity_df.to_csv('similarity.csv', index=True)
print("유사도 데이터프레임이 저장되었습니다.")

display(similarity_df)

유사도 데이터프레임이 저장되었습니다.


,aljazeera,alternet,americanconservative,ap,atlantic,bbc,breitbart,businessinsider,buzzfeed,cbs,...,slate,spectator,techcrunch,time,usatoday,verge,vice,vox,wapo,wsj
aljazeera,NaN,71.581031,71.845759,74.146983,72.478437,57.261125,70.87358,66.726684,70.036632,70.757859,...,74.441954,60.853315,82.192921,72.940619,75.9753,61.625865,66.420992,82.259769,72.096392,61.246512
alternet,71.581031,NaN,57.261125,70.87358,66.726684,70.036632,70.757859,77.276074,70.958759,70.803117,...,73.212529,77.195552,62.705971,66.420992,81.595132,72.416341,64.376183,70.422433,65.973309,72.890177
americanconservative,71.845759,57.261125,NaN,70.036632,70.757859,77.276074,70.958759,70.803117,75.517134,78.593491,...,62.705971,67.761274,82.623516,72.416341,64.376183,70.022023,64.791037,71.180332,64.341477,59.577297
ap,74.146983,70.87358,70.036632,NaN,77.276074,70.958759,70.803117,75.517134,78.593491,71.931271,...,67.761274,82.623516,73.685438,64.376183,70.022023,64.791037,70.721622,63.186699,59.168052,60.266581
atlantic,72.478437,66.726684,70.757859,77.276074,NaN,70.958759,70.803117,75.517134,78.593491,71.931271,...,67.761274,82.623516,73.685438,64.376183,70.022023,64.791037,70.721622,63.186699,59.168052,60.266581
bbc,57.261125,70.036632,77.276074,70.958759,70.958759,NaN,70.958759,70.803117,75.517134,78.593491,...,62.705971,67.761274,82.623516,72.416341,64.376183,70.022023,64.791037,71.180332,64.341477,59.577297
breitbart,70.87358,70.757859,70.958759,70.803117,70.803117,70.958759,NaN,77.276074,70.958759,70.803117,...,73.212529,77.195552,62.705971,66.420992,81.595132,72.416341,64.376183,70.422433,65.973309,72.890177
businessinsider,66.726684,77.276074,70.803117,75.517134,75.517134,70.803117,77.276074,NaN,70.036632,70.757859,...,74.441954,60.853315,82.192921,72.940619,75.9753,61.625865,66.420992,82.259769,72.096392,61.246512
buzzfeed,70.036632,70.958759,75.517134,78.593491,78.593491,75.517134,70.958759,70.036632,NaN,57.261125,...,81.209523,74.361702,71.213768,67.323187,73.669868,59.630696,80.823146,75.073959,75.101308,61.053262
cbs,70.757859,70.803117,78.593491,71.931271,71.931271,78.593491,70.803117,70.757859,57.261125,NaN,...,66.874192,72.968679,63.595406,75.23909,72.789515,81.022602,74.040033,72.484606,66.552739,74.716128
